In [1]:
# Instalar Java Tool Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar Spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompactar Spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

# Instalar lib findspark que ajuda a localizar o Spark no sistema e importa-lo como uma biblioteca regular
!pip install -q findspark

In [2]:
# Lib os - Permite Interagir com S.O - Cria variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [3]:
# Inicializa Findspark - Facilita Uso de Spark em Notebook
import findspark
findspark.init()

In [4]:
# Start SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName('Structured Stream')\
        .master('local')\
        .getOrCreate()

In [5]:
# Define Schema de Dataframe
from pyspark.sql.types import *

esquema = StructType([StructField('id', IntegerType(), True),
                      StructField('nome', StringType(), True),
                      StructField('idade', IntegerType(), True),
                      StructField('profissao', StringType(), True),
                      StructField('cidade', StringType(), True),
                      StructField('salario', DoubleType(), True)])

In [6]:
# Cria Dataframe a partir de Fluxo Continuo de Dados
df_pessoas = spark.readStream.\
             option('sep', ',').\
             option('header', 'true').\
             schema(esquema).\
             csv('Streaming')

In [26]:
# Confirma Objeto ser Streaming e Apresenta Descrição do Tipo de Objeto
print(df_pessoas.isStreaming,'para Streaming - Tipo:',type(df_pessoas))

True para Streaming - Tipo: <class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
# Print Schema
df_pessoas.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- profissao: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- salario: double (nullable = true)



In [9]:
# Gera Dataframe df_profissao como uma agregacao de dados do Dataframe df_pessoas
from pyspark.sql.functions import *

df_profissao = df_pessoas.\
               groupBy('profissao').\
               agg((avg('salario').alias('salario_medio')),(count('id').alias('qt_pessoas')))

In [27]:
# Consulta Tipo de Objeto
type(df_profissao)

pyspark.sql.dataframe.DataFrame

In [10]:
# Coleta Dados Objeto de Consulta StreamingQuery DadosProfissoes
query = df_profissao \
        .writeStream \
        .format('memory') \
        .outputMode('complete') \
        .queryName('DadosProfissoes') \
        .start()

In [24]:
# Consulta StreamingQuery em Fluxo de Dados
# Nota: Consulte cada atualização de dados que passar pelo Fluxo
spark.sql("select profissao,round(salario_medio,2) salario_medio,qt_pessoas from DadosProfissoes").show(truncate=False)

+--------------------+-------------+----------+
|profissao           |salario_medio|qt_pessoas|
+--------------------+-------------+----------+
|'Programador'       |4914.0       |3         |
|'Analista Fiscal'   |1558.0       |1         |
|'Motorista'         |6325.0       |2         |
|'Medico'            |7326.33      |3         |
|'Corretor'          |2100.0       |1         |
|'Enfermeiro'        |3522.0       |1         |
|'Cientista de Dados'|7612.0       |1         |
|'Designer'          |4607.0       |2         |
|'Carteiro'          |7612.0       |1         |
|'Professor'         |2090.67      |3         |
|'Ator'              |5667.0       |1         |
|'Policial'          |2263.0       |1         |
+--------------------+-------------+----------+



In [25]:
# Confirmando total de pessoas (linhas) processadas
spark.sql("select sum(qt_pessoas) from DadosProfissoes").show(truncate=False)

+---------------+
|sum(qt_pessoas)|
+---------------+
|20             |
+---------------+



In [92]:
# Parar SparkContext - Somente Executar se Precisar Reprocessar o Streaming
spark.stop()

In [73]:
# Excluir Pasta com Conteudo, caso seja necessario para Reprocessar o Streaming
!rm -rf /content/Streaming